In [1]:
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
!pip install -r requirements.txt  

Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17516 (delta 6), reused 0 (delta 0), pack-reused 17497 (from 4)
Receiving objects: 100% (17516/17516), 16.61 MiB | 29.94 MiB/s, done.
Resolving deltas: 100% (11994/11994), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:

In [2]:
!python classify/train.py \
  --model yolov5s-cls.pt \
  --data /kaggle/input/rbflow2-accident-dataset/rbflow2_accident-dataset \
  --epochs 50 \
  --imgsz 224 \
  --lr 0.0001 \
  --batch-size 16 \
  --cache ram \
  --workers 2 


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-15 09:44:12.671761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752572653.006275      82 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752572653.117518      82 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [3]:
!python classify/val.py \
  --weights runs/train-cls/exp/weights/best.pt \
  --data /kaggle/input/rbflow2-accident-dataset/rbflow2_accident-dataset \
  --imgsz 224

classify/val: data=/kaggle/input/rbflow2-accident-dataset/rbflow2_accident-dataset, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4169250 parameters, 0 gradients, 10.4 GFLOPs
testing:   0%|          | 0/13 [00:00<?, ?it/s]/kaggle/working/yolov5/classify/val.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type != "cpu"):
testing: 100%|██████████| 13/13 [00:02<00:00,  5.01it/s]
                   Class      Images    top1_acc    top5_acc
                     all        1561       0.919           1
                Accident         716       0.892           1
             NonAccident         845     

In [4]:
from utils.plots import plot_results

plot_results("runs/train-cls/exp/results.csv")

In [5]:
import torch
from pathlib import Path
from torchvision import datasets, transforms
import os
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay, accuracy_score,
    precision_recall_curve, average_precision_score
)
import pandas as pd
import numpy as np

# === Load model ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train-cls/exp/weights/best.pt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

# === Prepare dataset ===
#data_dir = '/kaggle/input/accident-test2/test2'
data_dir = '/kaggle/input/rbflow2-accident-dataset/rbflow2_accident-dataset/test'
#data_dir = '/kaggle/input/rbflow-accident-dataset/rbflow_accident_dataset/test'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
dataset = datasets.ImageFolder(data_dir, transform=transform)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

# === Prediction ===
y_true = []
y_pred = []
y_prob = []  # Save class 0 ("Accident") probabilities

threshold = 0.3
with torch.no_grad():
    for img, label in loader:
        img = img.to(device)
        logits = model(img)
        probs = torch.nn.functional.softmax(logits, dim=1).cpu()

        prob_accident = probs[0][0].item()  # class 0 = 'Accident'
        pred_label = 0 if prob_accident > threshold else 1  # binary threshold decision

        y_pred.append(dataset.classes[pred_label])
        y_true.append(dataset.classes[label.item()])
        y_prob.append(prob_accident)

# === Confusion Matrix ===
cm = confusion_matrix(y_true, y_pred, labels=dataset.classes)
acc = accuracy_score(y_true, y_pred)

# Plot Confusion Matrix
fig, ax = plt.subplots(figsize=(6, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dataset.classes)
disp.plot(cmap=plt.cm.Blues, values_format='d', ax=ax)
plt.title(f"Confusion Matrix (Accuracy: {acc:.2%})")
plt.xlabel("Predicted")
plt.ylabel("Actual")

# Save confusion matrix
save_path = 'runs/val-cls/exp'
os.makedirs(save_path, exist_ok=True)
plt.savefig(os.path.join(save_path, 'confusion_matrix.png'))
plt.show()

# === Precision-Recall Curve ===
y_true_binary = [0 if cls == 'Accident' else 1 for cls in y_true]  # class 0 = Accident
precision, recall, thresholds = precision_recall_curve(y_true_binary, y_prob)
ap_score = average_precision_score(y_true_binary, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'AP = {ap_score:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (for "Accident" Class)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(save_path, 'precision_recall_curve.png'))
plt.show()

# === Accuracy & Loss Plotting ===
results_path = Path('runs/train-cls/exp/results.csv')
if results_path.exists():
    df = pd.read_csv(results_path)

    # Plot Loss
    plt.figure(figsize=(8, 4))
    plt.plot(df.iloc[:, 0], df.iloc[:, 1], label='Train Loss', marker='o')
    plt.plot(df.iloc[:, 0], df.iloc[:, 2], label='Validation Loss', marker='s')
    plt.title("Loss per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, 'loss_plot.png'))
    plt.show()

    # Plot Accuracy
    plt.figure(figsize=(8, 4))
    plt.plot(df.iloc[:, 0], df.iloc[:, 3], label='Validation Accuracy', marker='o')  # Adjust index as needed
    plt.title("Validation Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, 'val_accuracy_plot.png'))
    plt.show()
else:
    print(f"results.csv not found at: {results_path}")


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4169250 parameters, 0 gradients, 10.4 GFLOPs
WARNING ⚠️ YOLOv5 ClassificationModel is not yet AutoShape compatible

In [6]:
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, _ = roc_curve(y_true_binary, y_prob)
auc_score = roc_auc_score(y_true_binary, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], '--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(save_path, 'roc_curve.png'))
plt.show()


In [7]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_true, y_pred, target_names=dataset.classes, digits=4)
print("📊 Classification Report:\n")
print(report)

# Optional: Save to file
with open(os.path.join(save_path, "classification_report.txt"), "w") as f:
    f.write("Classification Report\n")
    f.write(report)


📊 Classification Report:

              precision    recall  f1-score   support

    Accident     0.9383    0.5307    0.6780       716
 NonAccident     0.7093    0.9704    0.8196       845

    accuracy                         0.7687      1561
   macro avg     0.8238    0.7506    0.7488      1561
weighted avg     0.8143    0.7687    0.7546      1561



In [8]:
!python classify/train.py \
  --model yolov5s-cls.pt \
  --data /kaggle/input/finetunedataset/fineTuneDataset \
  --epochs 20 \
  --imgsz 224 \
  --lr 0.0001 \
  --batch-size 16 \
  --cache ram \
  --workers 2 

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-15 10:10:32.586871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752574232.609956     188 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752574232.616965     188 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&